In [5]:
import openai
import os
import json
import sys
from collections import defaultdict
#from dotenv import load_dotenv

#load_dotenv()

# OpenAI API 키 설정
openai.api_key = os.getenv('OPENAI_API_KEY')

# JSON 파일 로드
with open('./DataSet/User_DataSet.json', 'r') as f:
    data = json.load(f)

# 사용자 이름으로 데이터를 검색하는 함수
def find_user_data(name, data):
    for user in data:
        if user['name'] == name:
            return user
    return None

# 소비 패턴 분석 함수
def analyze_spending(transactions):
    spending = defaultdict(int)
    for transaction in transactions:
        category = transaction[3]
        amount = int(transaction[2])
        spending[category] += amount
    return spending

# 카드 추천 함수
def recommend_card_gpt(spending, user_name):
    spending_summary = "\n".join([f"{category}: {amount}원" for category, amount in spending.items()])

    prompt = (
        f"{user_name}의 소비 패턴은 다음과 같습니다:\n"
        f"{spending_summary}\n\n"
        "이 소비 패턴을 기반으로 추천할 만한 신용카드는 무엇인가요?"
    )

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant who recommends credit cards based on spending patterns."},
            {"role": "user", "content": prompt}
        ]
    )

    return response['choices'][0]['message']['content']

# 카테고리, 소비패턴에 따른 카드 추천
def recommend_card_based_on_input(user_input, user_name, user_data):
    # 소비 패턴 분석
    transactions = user_data.get('transactions_Withdrawal', [])
    spending = analyze_spending(transactions)
    spending_summary = "\n".join([f"{category}: {amount}원" for category, amount in spending.items()])

    prompt = (
        f"{user_name}님이 다음과 같은 질문을 하셨습니다:\n"
        f"{user_input}\n\n"
        "이 질문이 신용카드 추천과 관련이 있다면, "
        "이 소비 패턴을 기반으로 추천할 만한 신용카드를 알려주세요.\n"
        f"소비 패턴은 다음과 같습니다:\n"
        f"{spending_summary}"
    )

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a knowledgeable assistant who provides recommendations based on user input and spending patterns."},
            {"role": "user", "content": prompt}
        ]
    )
    
    return response['choices'][0]['message']['content']

# 일반 질문 처리
def gpt_general_response(user_input):
    # 열받는 기본 출력 문장 삭제
    prompt = f"{user_input}\nAnswer concisely and directly without any preamble."
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message['content']

# 입력에 따른 정보 처리
def handle_user_input(user_name):
    user_data = find_user_data(user_name, data)

    if not user_data:
        return f"{user_name}님의 데이터를 찾을 수 없습니다."

    while True:
        user_input = input(f"{user_name}님, 질문을 입력하세요 ('종료' 입력 시 종료): ")

        if user_input.lower() == '종료':
            print("대화를 종료합니다.")
            break

        # 카테고리 및 카드 추천 처리
        recommendation = recommend_card_based_on_input(user_input, user_name, user_data)

        if "추천" in recommendation or "카드" in recommendation:
            print(recommendation)
        else:
            # 일반 질문 처리
            response = gpt_general_response(user_input)
            print(response)

        sys.stdout.flush()

# 사용 시작
user_name = input("사용자의 이름을 입력하세요: ")
handle_user_input(user_name)

# 버퍼를 비웠음에도 입력 후 공백 답변 혹은 다음 답변을 주어야 제대로 된 기존 질문 답변 출력됨

주어진 소비 패턴을 고려하여 추천해드릴만한 카드는 현대카드 M포인트 카드입니다. 이 카드는 다양한 소비 카테고리에서 큰 혜택을 제공하며, 특히 스포츠, 영화, 엔터테인먼트, 음식점 및 여행에 대해 다양한 혜택을 제공합니다. 또한 교육 관련 지출에 대해서도 적극적인 혜택을 제공하여 다양한 활동을 지원해줍니다. 최윤호님의 다양한 소비 습관을 고려할 때 현대카드 M포인트 카드가 좋은 선택이 될 것입니다.
안정적인 투자를 원하시는 경우, 저-risk 투자 상품을 고려하실 수 있습니다. 예를 들어, CD (Certificate of Deposit), 국채, 또는 상장지수 펀드 등이 안정적인 투자 옵션입니다. 물론, 투자에 대한 전문적인 자문을 받으시는 것이 좋습니다.

또한, 신용카드 추천을 원하신다면, 주로 Health, Education, OTT, Travel에 지출하는 것으로 보이며, 이를 고려하여 혜택이 높은 카드 중에서 카테고리별 혜택이나 리워드를 많이 주는 카드를 선택하시는 것이 좋습니다. 혜택적인 면과 연간 수수료, APR 등을 고려하여 카드를 선택하시면 소비패턴에 더 유리한 카드를 고를 수 있을 것입니다.
신용카드를 추천하기 전에 공격적인 투자 방법에 대해 간단히 설명해 드리겠습니다. 

공격적인 투자 방법은 높은 위험을 감수하고 높은 수익을 얻기 위해 단기적으로 주식이나 다른 투자 상품에 대규모로 투자하는 것을 말합니다. 이 방법은 시장 변동성이 크고 단기적인 수익을 원하는 투자자에게 적합합니다. 하지만 주의할 점은 손실 가능성이 크고 금융 상황에 민감하다는 것입니다.

이제 여러분의 소비 패턴을 고려하여 적합한 신용카드를 추천해 드리겠습니다. 주로 사용하는 카테고리와 지출액을 고려해 특정 혜택이나 리워드를 제공하는 카드를 추천할 수 있습니다.

주로 사용하는 카테고리와 지출액을 고려해 적합한 카드를 추천해드리려면 최근 사용량과 소비 성향에 대한 정보가 필요합니다. 더 자세한 정보를 제공해 주시면 더 정확한 추천을 드릴 수 있습니다.
대화를 종료합니다.